In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import gymnasium as gym

env = gym.make("CartPole-v1", render_mode="rgb_array")

In [4]:
env.reset()
image = env.render()

In [ ]:
plt.figure(figsize=(8, 6))
plt.ion()  # Turn on interactive mode

for _ in range(100):  # Run for 100 steps
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    image = env.render()
    
    plt.clf()
    plt.imshow(image)
    plt.axis('off')
    plt.pause(0.01)
    
    if terminated or truncated:
        env.reset()

plt.ioff()
plt.show()